In [2]:
import torch 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score

In [ ]:
np.random.seed(1234)  
PYTHONHASHSEED = 0

In [ ]:
train_df = pd.read_csv('/Users/jjburrell/Downloads/PM_train.txt', sep='\s+', header=None)
test_df = pd.read_csv('/Users/jjburrell/Downloads/PM_test.txt', sep='\s+', header=None)

columns = [
    'unit_number', 
    'cycle', 
    'operational_setting_1',
    'operational_setting_2',
    'operational_setting_3',
    'fan_inlet_temperature',
    'lpc_pressure_ratio',
    'hpc_pressure_ratio',
    'lpt_temperature',
    'hpt_temperature',
    'fuel_flow_rate',
    'oil_pressure',
    'vibration_level',
    'exhaust_gas_temperature',
    'shaft_speed',
    'ambient_air_pressure',
    'ambient_air_temperature',
    'total_air_pressure',
    'static_pressure_ratio',
    'torque',
    'acceleration',
    'compressor_discharge_temperature',
    'combustion_chamber_pressure',
    'fuel_temperature',
    'nozzle_pressure_ratio',
    'oil_temperature'
]

# Apply column names
train_df.columns = columns
test_df.columns = columns

<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
/var/folders/kh/hkkfd68s6mb8b11z27mn1b4h0000gn/T/ipykernel_87117/654370591.py:4: SyntaxWarning: invalid escape sequence '\s'
  train_df = pd.read_csv('/Users/jjburrell/Downloads/PM_train.txt', sep='\s+', header=None)
/var/folders/kh/hkkfd68s6mb8b11z27mn1b4h0000gn/T/ipykernel_87117/654370591.py:5: SyntaxWarning: invalid escape sequence '\s'
  test_df = pd.read_csv('/Users/jjburrell/Downloads/PM_test.txt', sep='\s+', header=None)


   unit_number  cycle  operational_setting_1  operational_setting_2  \
0            1      1                -0.0007                -0.0004   
1            1      2                 0.0019                -0.0003   
2            1      3                -0.0043                 0.0003   
3            1      4                 0.0007                 0.0000   
4            1      5                -0.0019                -0.0002   

   operational_setting_3  fan_inlet_temperature  lpc_pressure_ratio  \
0                  100.0                 518.67              641.82   
1                  100.0                 518.67              642.15   
2                  100.0                 518.67              642.35   
3                  100.0                 518.67              642.35   
4                  100.0                 518.67              642.37   

   hpc_pressure_ratio  lpt_temperature  hpt_temperature  ...  \
0             1589.70          1400.60            14.62  ...   
1             1591

In [4]:
train_df = pd.read_csv('/Users/jjburrell/Downloads/PM_train.txt', sep=" ", header=None)

# Test data where the failure point is not given for the engines
test_df = pd.read_csv('/Users/jjburrell/Downloads/PM_test.txt', sep=" ", header=None)

# We wil print some rows of the train data
train_df.head(100)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,NaN,NaN
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,NaN,NaN
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,NaN,NaN
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,NaN,NaN
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,96,-0.0034,0.0001,100.0,518.67,642.19,1584.07,1395.16,14.62,...,8130.69,8.4311,0.03,392,2388,100.0,38.88,23.3255,NaN,NaN
96,1,97,0.0035,-0.0003,100.0,518.67,642.07,1595.77,1407.81,14.62,...,8128.74,8.4105,0.03,392,2388,100.0,39.01,23.2963,NaN,NaN
97,1,98,0.0006,0.0004,100.0,518.67,642.00,1591.11,1404.56,14.62,...,8127.89,8.4012,0.03,391,2388,100.0,38.96,23.2554,NaN,NaN
98,1,99,-0.0005,-0.0000,100.0,518.67,642.46,1592.73,1406.13,14.62,...,8131.77,8.4481,0.03,393,2388,100.0,38.82,23.2323,NaN,NaN


In [5]:
train_df.dropna(axis=1, inplace=True)
test_df.dropna(axis=1, inplace=True)

In [6]:
print(len(train_df))
print(len(test_df))

20631
13096


Section

In [9]:
import pandas as pd
import numpy as np

# Load train data
train_df = pd.read_csv('/Users/jjburrell/Downloads/PM_train.txt', sep='\s+', header=None)

# Assign column names
columns = [
    'unit_number', 'cycle', 'setting_1', 'setting_2', 'setting_3',
    'fan_inlet_temperature', 'lpc_pressure_ratio', 'hpc_pressure_ratio',
    'lpt_temperature', 'hpt_temperature', 'fuel_flow_rate', 'oil_pressure',
    'vibration_level', 'exhaust_gas_temperature', 'shaft_speed',
    'ambient_air_pressure', 'ambient_air_temperature', 'total_air_pressure',
    'static_pressure_ratio', 'torque', 'acceleration',
    'compressor_discharge_temperature', 'combustion_chamber_pressure',
    'fuel_temperature', 'nozzle_pressure_ratio', 'oil_temperature'
]
train_df.columns = columns

# Calculate RUL
rul_df = train_df.groupby('unit_number')['cycle'].max().reset_index()
rul_df.columns = ['unit_number', 'max_cycle']
train_df = train_df.merge(rul_df, on='unit_number', how='left')
train_df['RUL'] = train_df['max_cycle'] - train_df['cycle']
train_df.drop('max_cycle', axis=1, inplace=True)

print(train_df.head())

   unit_number  cycle  setting_1  setting_2  setting_3  fan_inlet_temperature  \
0            1      1    -0.0007    -0.0004      100.0                 518.67   
1            1      2     0.0019    -0.0003      100.0                 518.67   
2            1      3    -0.0043     0.0003      100.0                 518.67   
3            1      4     0.0007     0.0000      100.0                 518.67   
4            1      5    -0.0019    -0.0002      100.0                 518.67   

   lpc_pressure_ratio  hpc_pressure_ratio  lpt_temperature  hpt_temperature  \
0              641.82             1589.70          1400.60            14.62   
1              642.15             1591.82          1403.14            14.62   
2              642.35             1587.99          1404.20            14.62   
3              642.35             1582.79          1401.87            14.62   
4              642.37             1582.85          1406.22            14.62   

   ...  total_air_pressure  static_pre

<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
/var/folders/kh/hkkfd68s6mb8b11z27mn1b4h0000gn/T/ipykernel_87117/4252076313.py:5: SyntaxWarning: invalid escape sequence '\s'
  train_df = pd.read_csv('/Users/jjburrell/Downloads/PM_train.txt', sep='\s+', header=None)


In [10]:
# Feature columns for traditional ML
feature_columns = columns[2:]  # exclude unit_number, cycle

X = train_df[feature_columns].values
y = train_df['RUL'].values

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_val)
print("Random Forest RMSE:", np.sqrt(mean_squared_error(y_val, y_pred_rf)))

Random Forest RMSE: 41.5214287563132


In [14]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_val)
print("XGBoost RMSE:", np.sqrt(mean_squared_error(y_val, y_pred_xgb)))

XGBoost RMSE: 43.41286629523437


In [15]:
from sklearn.linear_model import ElasticNet

enet = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
enet.fit(X_train, y_train)
y_pred_enet = enet.predict(X_val)
print("ElasticNet RMSE:", np.sqrt(mean_squared_error(y_val, y_pred_enet)))

ElasticNet RMSE: 44.85389304111896


In [16]:
sequence_length = 50

def generate_sequences(df, sequence_length):
    sequences, rul_values = [], []
    for unit in df['unit_number'].unique():
        unit_data = df[df['unit_number'] == unit].reset_index(drop=True)
        for start in range(len(unit_data) - sequence_length + 1):
            seq = unit_data.loc[start:start+sequence_length-1, feature_columns].values
            sequences.append(seq)
            rul_values.append(unit_data.loc[start+sequence_length-1, 'RUL'])
    return np.array(sequences), np.array(rul_values)

X_seq, y_seq = generate_sequences(train_df, sequence_length)

X_seq_train, X_seq_val, y_seq_train, y_seq_val = train_test_split(
    X_seq, y_seq, test_size=0.2, random_state=42)


In [17]:
import torch
from torch.utils.data import Dataset, DataLoader

class CmapssDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = CmapssDataset(X_seq_train, y_seq_train)
val_dataset = CmapssDataset(X_seq_val, y_seq_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [18]:
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size=64, batch_first=True)
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out.squeeze()

class GRUModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.gru = nn.GRU(input_size, hidden_size=64, batch_first=True)
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        out, _ = self.gru(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out.squeeze()

class CNNModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.conv1 = nn.Conv1d(input_size, 64, kernel_size=3)
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = torch.relu(self.conv1(x))
        x = self.pool(x).squeeze(-1)
        x = self.fc(x)
        return x.squeeze()

In [19]:
def train_model(model, train_loader, val_loader, num_epochs=100):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

        if (epoch+1) % 10 == 0 or epoch == 0:
            model.eval()
            val_losses = []
            with torch.no_grad():
                for X_batch, y_batch in val_loader:
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    outputs = model(X_batch)
                    val_loss = criterion(outputs, y_batch)
                    val_losses.append(val_loss.item())
            print(f"Epoch [{epoch+1}/{num_epochs}], Validation RMSE: {np.sqrt(np.mean(val_losses)):.4f}")


In [20]:
model_lstm = LSTMModel(input_size=len(feature_columns))
train_model(model_lstm, train_loader, val_loader, num_epochs=100)

Epoch [1/100], Validation RMSE: 95.5563
Epoch [10/100], Validation RMSE: 63.5302
Epoch [20/100], Validation RMSE: 57.1019
Epoch [30/100], Validation RMSE: 57.1362
Epoch [40/100], Validation RMSE: 57.1337
Epoch [50/100], Validation RMSE: 57.1363
Epoch [60/100], Validation RMSE: 57.1367
Epoch [70/100], Validation RMSE: 57.1347
Epoch [80/100], Validation RMSE: 57.1366
Epoch [90/100], Validation RMSE: 57.1348
Epoch [100/100], Validation RMSE: 57.1339


In [22]:
model_gru = GRUModel(input_size=len(feature_columns))
train_model(model_gru, train_loader, val_loader, num_epochs=100)

Epoch [1/100], Validation RMSE: 91.5074
Epoch [10/100], Validation RMSE: 57.2825
Epoch [20/100], Validation RMSE: 57.1350
Epoch [30/100], Validation RMSE: 57.1263
Epoch [40/100], Validation RMSE: 57.1263
Epoch [50/100], Validation RMSE: 57.1311
Epoch [60/100], Validation RMSE: 57.1320
Epoch [70/100], Validation RMSE: 57.1357
Epoch [80/100], Validation RMSE: 57.1361
Epoch [90/100], Validation RMSE: 57.1283
Epoch [100/100], Validation RMSE: 57.1368
